# YOLO v5

### 0. Install Yolov5

In [3]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 1576 (delta 21), reused 32 (delta 13), pack-reused 1532
Receiving objects: 100% (1576/1576), 4.56 MiB | 2.72 MiB/s, done.
Resolving deltas: 100% (1052/1052), done.


In [4]:
! cd yolov5 && pip install -U -r requirements.txt

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-uqcnc51b
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-uqcnc51b
Requirement already up-to-date: Cython in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 2)) (0.29.21)
     |████████████████████████████████| 20.0MB 1.4MB/s 
     |████████████████████████████████| 43.7MB 139kB/s 
Requirement already up-to-date: torch>=1.5.1 in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 5)) (1.5.1+cu101)
Requirement already up-to-date: matplotlib in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 6)) (3.2.2)
     |████████████████████████████████| 2.2MB 44.0MB/s 
Requirement already up-to-date: tensorboard in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 8)) (2.2.2)
     |████████████████████████████████| 276kB 62.9MB/s 
Requirement already up-to-date: torchvision in /usr/local/lib/pytho

### 1. Test out model on video input

In [5]:
!wget http://capslock.ai/drive.mp4

--2020-07-16 16:28:20--  http://capslock.ai/drive.mp4
Resolving capslock.ai (capslock.ai)... 74.125.24.128, 2404:6800:4003:c02::80
Connecting to capslock.ai (capslock.ai)|74.125.24.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18170896 (17M) [video/mp4]
Saving to: ‘drive.mp4’

drive.mp4           100%[===================>]  17.33M  9.60MB/s    in 1.8s    

2020-07-16 16:28:23 (9.60 MB/s) - ‘drive.mp4’ saved [18170896/18170896]



In [6]:
!cd yolov5 && python detect.py --source ../drive.mp4

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', img_size=640, iou_thres=0.5, output='inference/output', save_txt=False, source='../drive.mp4', update=False, view_img=False, weights='yolov5s.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   279    0   279    0     0   1334      0 --:--:-- --:--:-- --:--:--  1334
100   408    0   408    0     0    519      0 --:--:-- --:--:-- --:--:--  1096
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 14.4M    0 14.4M    0     0  8667k      0 --:--:--  0:00:01 --:--:--  108M
Fusing layers... Model Summary: 140 layers, 7.45958e+06 parameters, 7.45958e+06 gradients
video 1/1 (1/1648) /content/drive.mp4: 448x

### 2. Export model to onnx

In [7]:
!pip install onnx

     |████████████████████████████████| 7.4MB 17.4MB/s 


In [15]:
!ls yolov5/models
!cat yolov5/models/export.py
!cd yolov5 && python models/export.py

common.py	 hub	      yolo.py	    yolov5s.yaml
experimental.py  __init__.py  yolov5l.yaml  yolov5x.yaml
export.py	 __pycache__  yolov5m.yaml
"""Exports a YOLOv5 *.pt model to ONNX and TorchScript formats

Usage:
    $ export PYTHONPATH="$PWD" && python models/export.py --weights ./weights/yolov5s.pt --img 640 --batch 1
"""

import sys
import os 

print('cwd:', os.getcwd())
sys.path.append(os.getcwd())

import argparse

from models.common import *
from utils import google_utils

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', type=str, default='./yolov5s.pt', help='weights path')
    parser.add_argument('--img-size', nargs='+', type=int, default=[640, 640], help='image size')
    parser.add_argument('--batch-size', type=int, default=1, help='batch size')
    opt = parser.parse_args()
    opt.img_size *= 2 if len(opt.img_size) == 1 else 1  # expand
    print(opt)

    # Input
    img = torch.zeros((opt.batch_size, 3, *opt.img_size))  

### 3. Simplyify model architecture

In [16]:
!pip install onnx-simplifier

     |████████████████████████████████| 3.9MB 16.0MB/s 
  Created wheel for onnx-simplifier: filename=onnx_simplifier-0.2.10-cp36-none-any.whl size=6499 sha256=e7a44bc921ef66631c4900048bc1077fa76e7f70c3483b26b39eb536ff41ed9d
  Stored in directory: /root/.cache/pip/wheels/a0/6b/aa/b3b099f96c24c783a74af4bd1b051b526ff927505e3ffc9f60
Successfully built onnx-simplifier


In [17]:
!cd yolov5 && python3 -m onnxsim yolov5s.onnx yolov5s-simple.onnx

Simplifying...
Checking 0/3...
Checking 1/3...
Checking 2/3...
Ok!
